# 코사인 유사도
- 오늘은 유사도 검사에 대해 다룰 것
- 텍스트 마이닝 후에는 유사도 검사가 처음할 것
- 그 다음은 토픽 모델링 ( 주로 영어갖고 많이 한다)

## 문서 유사도

- 단어를 컴퓨터가 알아들을 수 있도록 정수 인코딩 및 실수 부여를 해줘야함
- 문장도 ranking 가능 (나는 꽃을 좋아한다 / 당신은 꽃을 싫어한다 -> 모든 단어로 BOW 만든 후 10110, 01101 식으로 encoding 가능)
- 5차원 공간좌표임 (10 11 0) _ 벡터화한다고 한다.

### 유사도 측정방식
- 유클리드 거리
- 코사인유사도
- 맨해튼 거리
- 자카드 유사도

- 유클리드, 코사인은 모두 텍스트 마이닝과 이미지 마이닝에 적합하다고 서술되어있는 편. 데이터에 따른 유사도 검사는 직접 갖다 쓸 필요가 있다.

1. 코사인 유사도
- 두 벡터간의 코사인 각도를 이용하여 유사도를 측정하는 방법
- 여기서 벡터 값은 좌표라고 생각하면 편하다.
- 1이면 높은 유사도, 0이면 관련성이 없고, -1은 도메인 따라 해석이 달라질 수 있다
- 두벡터의 내적 / 두 벡터의 norm의 곱으로 구할 수 있다.
- 두 벡터의 내적은 두 벡터에 대응하는 성분을 곱한 다음 그 곱들을 모두 더한 값
- (a1,a2),(b1,b2)의 내적은 a1*b1 + a2*b2
- a*b = |a||b|


- 벡터의 norm : 유클리드 norm( = L2 norm), 맨헤턴 norm (L1 norm), 무한 norm (Maximum norm)

2. 유클리드 norm
- 각 벡터의 성분을 제곱한 후 모두 더해서 제곱근을 취한 값
- ||a||2 = sqr.a2 + ... + an2
- 유사도는 BOW만 있으면 이론적으로는 손으로도 계산할 수 있다 ㅎㅎ

### 코사인 유사도 계산 라이브러리

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from IPython.display import display


In [7]:
# 행과 열을 다 보여달라는 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [24]:
cv = CountVectorizer()

doc1 = "Cão mordido de cobra tem medo até de corda"
doc2 = "Não acordes o cão quando ele está dorimindo"
#doc3 =  "A ferida do cão dura-se com o pêlo do mesmo cão"
doc3 = "Cão mordido tem medo de corda"

docs = [doc1, doc2, doc3]

dtm = cv.fit_transform(docs)

In [25]:
names = cv.get_feature_names_out()
count_words = pd.DataFrame(dtm.toarray(), columns = names)
display(count_words)

,acordes,até,cobra,corda,cão,de,dorimindo,ele,está,medo,mordido,não,quando,tem
0,0,1,1,1,1,2,0,0,0,1,1,0,0,1
1,1,0,0,0,1,0,1,1,1,0,0,1,1,0
2,0,0,0,1,1,1,0,0,0,1,1,0,0,1


In [26]:
# 코사인 유사도 검사 함수
def cos_sin(A,B):
  # dot: (a1*b1)+,,,(an*bn)내적을 구해주는 수
  return dot(A,B) / (norm(A) * norm(B))

In [27]:
cos_sin(count_words.iloc[0], count_words.iloc[1])

0.11396057645963795

In [28]:
similarities = pd.DataFrame(index = [f'Doc{i+1}' for i in range(len(docs))],
                            columns = [f'Doc {i+1}' for i in range(len(docs))])

# 아직 연산전이라 NaN나옴
similarities

,Doc 1,Doc 2,Doc 3
Doc1,NaN,NaN,NaN
Doc2,NaN,NaN,NaN
Doc3,NaN,NaN,NaN


In [29]:
# 코랩 추천 코드
for i in range(len(docs)):
  for j in range(len(docs)):
    similarities.iloc[i,j] = cos_sin(count_words.iloc[i], count_words.iloc[j])
    print(similarities.iloc[i,j])

1.0
0.11396057645963795
0.8616404368553293
0.11396057645963795
0.9999999999999999
0.1543033499620919
0.8616404368553293
0.1543033499620919
1.0000000000000002


In [30]:
# 교수님 강의 코드
for i in range(len(docs)):
  for j in range(len(docs)):
    similarities.iloc[i,j] = cos_sin(dtm.toarray()[i], dtm.toarray()[j])
    print(similarities.iloc[i,j])

1.0
0.11396057645963795
0.8616404368553293
0.11396057645963795
0.9999999999999999
0.1543033499620919
0.8616404368553293
0.1543033499620919
1.0000000000000002


- 주대각선은 자기 자신이라 일치도가 1.0
- 11%~20% 정도의 일치도를 보인다
- doc3고치니 86%까지도 오른다
- 다만 이 벡터는 단순한 거리연산일 뿐, 의미상의 벡터 연산은 아직 구현이 어렵다!

In [31]:
similarities

,Doc 1,Doc 2,Doc 3
Doc1,1.0,0.113961,0.86164
Doc2,0.113961,1.0,0.154303
Doc3,0.86164,0.154303,1.0


- 사실 위의 연산을 다 함수화해준 sklearn의 cosine_similarities 라이브러리가 존재한다

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import display

In [35]:
cv = CountVectorizer()

In [36]:
doc1 = "Cão mordido de cobra tem medo até de corda"
doc2 = "Não acordes o cão quando ele está dorimindo"
doc3 =  "A ferida do cão dura-se com o pêlo do mesmo cão"

docs = [doc1, doc2, doc3]

In [37]:
# sparse vectorizer : 희소행렬 반환
dtm = cv.fit_transform(docs)

In [38]:
cos_sim = cosine_similarity(dtm.toarray(), dtm.toarray())
display(cos_sim)

array([[1.        , 0.11396058, 0.16116459],
       [0.11396058, 1.        , 0.20203051],
       [0.16116459, 0.20203051, 1.        ]])

In [40]:
cos_sim_df = pd.DataFrame(cos_sim, columns = [f'Doc{i+1}' for i in range(len(docs))],
                          index = [f'Doc{i+1}' for i in range(len(docs))])
cos_sim_df

,Doc1,Doc2,Doc3
Doc1,1.000000,0.113961,0.161165
Doc2,0.113961,1.000000,0.202031
Doc3,0.161165,0.202031,1.000000


In [45]:
cos_sim_df.iloc[1,1]

0.9999999999999997

# 숙제: 엑셀 테이블 하나 만들어두기
- 행 6개 정도 되는